1. Nettoyage
2. Suppression
3. Gestion des NA
4. Création de features

In [ ]:
import pandas as pd 

df = pd.read_csv('data_set/SBAnational.csv')
df.head(10)

### Selection des colonnes

In [ ]:
df.columns

In [ ]:
df=df[['City', 'State', 'Zip', 'Bank', 'BankState',
       'NAICS', 'ApprovalDate', 'ApprovalFY', 'Term', 'NoEmp', 'NewExist',
       'CreateJob', 'RetainedJob', 'FranchiseCode', 'UrbanRural', 'RevLineCr',
       'LowDoc', 'ChgOffDate', 'DisbursementDate', 'DisbursementGross',
       'BalanceGross', 'MIS_Status', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv']]

### Transformation des dates 

In [ ]:
# erreur de frappe '1976A' au lieu de 1976

filtre = df['ApprovalFY']=='1976A'

df.loc[filtre,['ApprovalFY']]='1976'

In [ ]:
# transformation des colonnes ApprovalDate et DisbursementDate

df['ApprovalDate']= pd.to_datetime(df['ApprovalDate'], format='%d-%b-%y', errors='coerce')
df['DisbursementDate']= pd.to_datetime(df['DisbursementDate'], format='%d-%b-%y', errors='coerce')
df['ChgOffDate']= pd.to_datetime(df['ChgOffDate'], format='%d-%b-%y', errors='coerce')

# transformation de la colonne ApprovalFY

df['ApprovalFY']= pd.to_datetime(df['ApprovalFY'],format='%Y')



### Transformation des montants

In [ ]:
colonnes_montant = ['DisbursementGross','BalanceGross','ChgOffPrinGr','GrAppv','SBA_Appv']

for colonne in colonnes_montant:
    df[colonne] = pd.to_numeric(df[colonne].str.replace('[\$,]', '', regex=True).str.replace(',', '', regex=False), errors='coerce')
    df[colonne] = df[colonne].round(2)

df.head()

### Suppression des NA dans la target

In [ ]:
print(len(df))
df= df.dropna(subset=['MIS_Status'], axis=0)
print(len(df))

### Gestion des Borrower city

In [ ]:
#remplacer les valeurs NaN par la valeur la plus fréquente selon le code Zip

cities = df.groupby('Zip')['City'].apply(lambda x: x.mode().iloc[0])

df['City'] = df.apply(lambda row: row['City'] if pd.notnull(row['City']) else cities[row['Zip']], axis=1)

df['City'].isnull().sum()

### Gestion Borrower State


In [ ]:
#remplacer les valeurs NaN par la valeur la plus fréquente selon 'City'

# Fonction pour obtenir le mode tout en traitant les cas spéciaux
def get_mode(series):
    try:
        return series.mode().iloc[0]
    except IndexError:
        return 'NA'

# Obtenir le mode pour chaque groupe 'Zip' dans la colonne 'State'
states = df.groupby('City')['State'].apply(get_mode).reset_index()
df = pd.merge(df, states, on='City', how='left', suffixes=('', '_Mode'))
df['State'] = df['State_Mode']
df = df.drop('State_Mode', axis=1)

### Gestion de Bank

In [ ]:

# Obtenir le mode pour chaque groupe 'Zip' dans la colonne 'Bank'
Banks = df.groupby('City')['Bank'].apply(get_mode).reset_index()
df = pd.merge(df, Banks, on='City', how='left', suffixes=('', '_Mode'))
df['Bank'] = df['Bank_Mode']
df = df.drop('Bank_Mode', axis=1)

df[df['Bank'].isnull()]

### Gestion Bank State


In [ ]:
#remplacer les valeurs NaN par la valeur la plus fréquente selon 'Bank'

# Obtenir le mode pour chaque groupe 'Bank' dans la colonne 'BankState'
BankStates = df.groupby('City')['BankState'].apply(get_mode).reset_index()
df = pd.merge(df, BankStates, on='City', how='left', suffixes=('', '_Mode'))
df['BankState'] = df['BankState_Mode']
df = df.drop('BankState_Mode', axis=1)

### Gestion ApprovalDate


In [ ]:
filtre= df['ApprovalDate'].isnull()
df[filtre]

### Atomisation NewExist

In [ ]:
filtre = df['NewExist'].isnull()
df=df.dropna(subset=['NewExist'], axis=0)

df['NewExist'].isnull().sum()

### Gestion RevLineCr

In [ ]:
#On garde les valeurs 'Y' et 'N' pour les autres on met la valeur '-' :
def modifier_valeur(valeur):
    if valeur not in ['Y', 'N']:
        return '-'
    else:
        return valeur

#on remplace les 0 par N et les [1, T] par Y
def modifier_valeur2(valeur):
    if valeur in ['N', '0']:
        return 'N'
    if valeur in ['Y', '1', 'T']:
        return 'Y'
    else:
        return pd.NA
    
df['RevLineCr'] = df['RevLineCr'].apply(modifier_valeur)

### Gestion des LowDoc
<= 150 000 alors Y
> 150 000 alors N

In [ ]:
# gestion des NaN

filtre1 = df['LowDoc'].isnull()

filtre2 = df['GrAppv']<= 150000
filtre3= df['GrAppv']>150000

df.loc[filtre1&filtre2,'LowDoc']='Y'
df.loc[filtre1&filtre3,'LowDoc'] = 'N'

# gestion des valeurs autre que Y et N


filtre= (df['LowDoc']!= 'Y') & (df['LowDoc']!='N')
display(df.loc[filtre,['LowDoc','GrAppv']])
df.loc[filtre&filtre2,'LowDoc']='Y'
df.loc[filtre&filtre3,'LowDoc']='N'
display(df.loc[filtre,['LowDoc','GrAppv']])

df['LowDoc'].unique()



### Atomisation radioactive des DisbursementDate

In [ ]:
df=df.dropna(subset=['DisbursementDate'],axis=0)
df['DisbursementDate'].isnull().sum()

### Traitement des ChgOffDate

In [ ]:
#suppression des NaN dont la target = 'CHGOFF'
#pour les target = 'P I F' on met la date la plus récente :

df = df.query("MIS_Status != 'CHGOFF' or ChgOffDate.notna()")
# df['ChgOffDate'].describe()
max_date = '13-Mar-10'
df['ChgOffDate'] = df['ChgOffDate'].fillna(max_date)

### NAICS
les valeurs avec 6 caractères sont tronquées : on ne garde que les 2 premiers chiffres

In [ ]:
df['NAICS'] = df['NAICS'].astype(str)
df['NAICS'] = df['NAICS'].apply(lambda x: x[:2])
df['NAICS'] = df['NAICS'].astype(int)

### Traitement des FranchiseCode

In [ ]:
# 1        636232
# 0        206923
df['FranchiseCode'] = pd.to_numeric(df['FranchiseCode'], errors='coerce')
df['FranchiseCode'] = df['FranchiseCode'].replace([0, 1], '0')
df['FranchiseCode'] = df['FranchiseCode'].replace(to_replace=df['FranchiseCode'].unique()[1:], value='1')
df['FranchiseCode'].value_counts()

### différence entre la création et les jobs gardés

In [ ]:
df['DiffJobs'] = df['RetainedJob'] - df['CreateJob']
dd = df[['RetainedJob', 'CreateJob', 'DiffJobs', 'MIS_Status']]

In [ ]:
dd.query("RetainedJob != 0")

### des survivants à l'atomisation ???

                               ________________
                          ____/ (  (    )   )  \___
                         /( (  (  )   _    ))  )   )\
                       ((     (   )(    )  )   (   )  )
                     ((/  ( _(   )   (   _) ) (  () )  )
                    ( (  ( (_)   ((    (   )  .((_ ) .  )_
                   ( (  )    (      (  )    )   ) . ) (   )
                  (  (   (  (   ) (  _  ( _) ).  ) . ) ) ( )
                  ( (  (   ) (  )   (  ))     ) _)(   )  )  )
                 ( (  ( \ ) (    (_  ( ) ( )  )   ) )  )) ( )
                  (  (   (  (   (_ ( ) ( _    )  ) (  )  )   )
                 ( (  ( (  (  )     (_  )  ) )  _)   ) _( ( )
                  ((  (   )(    (     _    )   _) _(_ (  (_ )
                   (_((__(_(__(( ( ( |  ) ) ) )_))__))_)___)
                   ((__)        \\||lll|l||///          \_))
                            (   /(/ (  )  ) )\   )
                          (    ( ( ( | | ) ) )\   )
                           (   /(| / ( )) ) ) )) )
                         (     ( ((((_(|)_)))))     )
                          (      ||\(|(|)|/||     )
                        (        |(||(||)||||        )
                          (     //|/l|||)|\\ \     )
                        (/ / //  /|//||||\\  \ \  \ _)
-------------------------------------------------------------------------------


In [ ]:
df.isnull().sum()

In [ ]:
df.drop(columns='ChgOffDate',inplace=True)
df.drop(columns='DisbursementDate',inplace=True)
df.drop(columns='DisbursementGross',inplace=True)
df.drop(columns='BalanceGross',inplace=True)
df.drop(columns='ChgOffPrinGr',inplace=True)
df.drop(columns='SBA_Appv',inplace=True)

In [ ]:
#creation d'un csv avec la nouvelle version

# df.to_csv('./data_set/SBAnational-EDA.csv')